In [4]:
import mysql.connector

# إعداد بيانات الاتصال
config = {
    'user': 'lemon_user',
    'password': '1357',
    'host': 'localhost',           # أو عنوان السيرفر
    'database': 'littlelemondm1',   # اسم قاعدة البيانات
    'raise_on_warnings': True
}

try:
    # إنشاء الاتصال
    connection = mysql.connector.connect(**config)
    print("تم الاتصال بقاعدة البيانات بنجاح!")

    # يمكنك تنفيذ استعلامات هنا
    cursor = connection.cursor()
    cursor.execute("SELECT DATABASE();")
    db = cursor.fetchone()
    print(f"متصل بقاعدة البيانات: {db}")

except mysql.connector.Error as err:
    print(f"حدث خطأ: {err}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("تم إغلاق الاتصال بقاعدة البيانات.")


تم الاتصال بقاعدة البيانات بنجاح!
متصل بقاعدة البيانات: ('littlelemondm1',)
تم إغلاق الاتصال بقاعدة البيانات.


In [5]:
import mysql.connector

# إعدادات الاتصال بقاعدة البيانات
config = {
    'user': 'lemon_user',
    'password': '1357',
    'host': 'localhost',
    'database': 'littlelemondm1'
}

try:
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()

    # 1. إنشاء الإجراء المخزن (أولاً حذف إذا كان موجود لتجنب الخطأ)
    cursor.execute("DROP PROCEDURE IF EXISTS GetMaxQuantity;")

    create_procedure_query = """
    CREATE PROCEDURE GetMaxQuantity()
    BEGIN
        SELECT MAX(Quantity) AS MaxQuantity FROM Orders;
    END
    """
    cursor.execute(create_procedure_query)
    print("تم إنشاء الإجراء المخزن GetMaxQuantity بنجاح!")

    # 2. استدعاء الإجراء
    cursor.callproc('GetMaxQuantity')

    # 3. جلب وعرض النتائج
    for result in cursor.stored_results():
        data = result.fetchall()
        for row in data:
            print(f"Maximum ordered quantity is: {row[0]}")

except mysql.connector.Error as err:
    print(f"حدث خطأ: {err}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()


تم إنشاء الإجراء المخزن GetMaxQuantity بنجاح!
Maximum ordered quantity is: 4


C:\Users\ROG\AppData\Local\Temp\ipykernel_21168\4238046171.py:31: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for result in cursor.stored_results():


In [12]:
import mysql.connector
from mysql.connector import pooling

# إعداد الاتصال باستخدام connection pool
dbconfig = {
    "user": "lemon_user",
    "password": "1357",
    "host": "localhost",
    "database": "littlelemondm1"
}

# إنشاء pool
pool = pooling.MySQLConnectionPool(pool_name="mypool",
                                   pool_size=5,
                                   **dbconfig)

# استدعاء الاتصال من الـ pool
connection = pool.get_connection()
cursor = connection.cursor()

# استعلام SQL من task 2
query = """
SELECT 
    ` customers`.CustomerID,
    ` customers`.FullName,
    Orders.OrderID,
    Orders.TotalCost,
    Menu.Name AS MenuName
FROM Orders
JOIN ` customers` ON Orders.CustomerID = ` customers`.CustomerID
JOIN Menu ON Orders.MenuID = Menu.MenuID
WHERE Orders.TotalCost > 150
ORDER BY Orders.TotalCost ASC;
"""


# تنفيذ الاستعلام
cursor.execute(query)

# جلب النتائج
results = cursor.fetchall()

# عرض النتائج
for row in results:
    print(f"CustomerID: {row[0]}, FullName: {row[1]}, OrderID: {row[2]}, "
          f"TotalCost: {row[3]}, MenuName: {row[4]}")

# إغلاق الاتصال
cursor.close()
connection.close()


CustomerID: 1, FullName: gh, OrderID: 1, TotalCost: 234, MenuName: rt
CustomerID: 2, FullName: nd, OrderID: 2, TotalCost: 546, MenuName: ff
CustomerID: 3, FullName: mn, OrderID: 3, TotalCost: 777, MenuName: fg


In [9]:
cursor.execute("SHOW TABLES")
for table in cursor.fetchall():
    print(table)


(' customers',)
('bookings',)
('menu',)
('orderdeliverystatus',)
('orders',)
('staff',)


In [13]:
import mysql.connector
from mysql.connector import pooling

# إعداد الاتصال باستخدام connection pool
dbconfig = {
    "user": "lemon_user",
    "password": "1357",
    "host": "localhost",
    "database": "littlelemondm1"
}

# إنشاء pool
pool = pooling.MySQLConnectionPool(pool_name="mypool",
                                   pool_size=5,
                                   **dbconfig)

# استدعاء الاتصال من الـ pool
connection = pool.get_connection()
cursor = connection.cursor()

try:
    # رقم الطلب الذي تريد حذفه
    order_id_to_cancel = 10  # غير هذا الرقم حسب الحاجة

    # استدعاء الإجراء المخزن وتمرير المعامل
    cursor.callproc('CancelOrder', [order_id_to_cancel])

    # تنفيذ التغييرات
    connection.commit()

    print(f"Order with OrderID = {order_id_to_cancel} has been cancelled successfully.")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # إغلاق المؤشر والاتصال
    cursor.close()
    connection.close()


Order with OrderID = 10 has been cancelled successfully.


In [15]:
import mysql.connector
from mysql.connector import pooling

# إعداد الاتصال باستخدام connection pool
dbconfig = {
    "user": "lemon_user",
    "password": "1357",
    "host": "localhost",
    "database": "littlelemondm1"
}

# إنشاء pool
pool = pooling.MySQLConnectionPool(pool_name="mypool",
                                   pool_size=5,
                                   **dbconfig)

# الحصول على اتصال من الـ pool
connection = pool.get_connection()
cursor = connection.cursor()

# ------------------ Task 2: استدعاء CheckBooking ------------------
def check_booking(booking_date, table_number):
    try:
        cursor.callproc("CheckBooking", (booking_date, table_number))
        for result in cursor.stored_results():
            print(f"📅 تحقق من الحجز: {result.fetchone()[0]}")
    except mysql.connector.Error as err:
        print("❌ خطأ في CheckBooking:", err)

# مثال على استخدام check_booking
check_booking('2022-10-10', 5)
check_booking('2022-10-15', 1)

# ------------------ Task 3: استدعاء AddValidBooking ------------------
def add_valid_booking(booking_date, table_number, customer_id):
    try:
        cursor.callproc("AddValidBooking", (booking_date, table_number, customer_id))
        for result in cursor.stored_results():
            print(f"📝 نتيجة الحجز: {result.fetchone()[0]}")
    except mysql.connector.Error as err:
        print("❌ خطأ في AddValidBooking:", err)

# مثال على استخدام add_valid_booking
add_valid_booking('2022-10-10', 5, 2)  # مفترض أن يفشل (الطاولة محجوزة)
add_valid_booking('2022-10-15', 1, 3)  # مفترض أن ينجح (الطاولة متاحة)

# إغلاق الاتصال
cursor.close()
connection.close()


📅 تحقق من الحجز: Table is already booked.
📅 تحقق من الحجز: Table is available.
📝 نتيجة الحجز: Booking failed. Table already booked.
❌ خطأ في AddValidBooking: 1364 (HY000): Field 'BookingID' doesn't have a default value


C:\Users\ROG\AppData\Local\Temp\ipykernel_21168\707146215.py:25: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for result in cursor.stored_results():
C:\Users\ROG\AppData\Local\Temp\ipykernel_21168\707146215.py:38: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for result in cursor.stored_results():


In [19]:
import os
print(os.getcwd())


C:\Users\ROG


In [20]:
os.getcwd()

'C:\\Users\\ROG'